In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

### Working with results of a ZenGarden run

This notebook shows some examples on how to handle the results of a `zen-garden` run. It uses the outputs of the tests as examples. Therefore, you need to run the tests via 
```
coverage run --source="zen_garden" -m pytest -v tests/testcases/run_test.py
```
to get the data.

### Reading in the results

The results can be read in using the `Results` class. This will automatically read in all relevant files, independent of the number of scenarios or if the optimization was run with perfect or myopic foresight. However, you should know what type of input you are reading in, as the output of some routines will depend on the type.

In [6]:
# This has no scenarios and perfect foresight
res_basic = Results("../outputs/test_1a/")
# Here we have three different scenarios
res_scenario = Results("../outputs/test_6a/")
# This was run with myopic foresight
res_mf = Results("../outputs/test_4g/")
# This was run with aggregated time steps and multi year
res_ts = Results("../outputs/test_4f/")

### Accessing the config

The standard dictionaries, i.e. system, analysis, scenarios, solver and dictSequenceTimeSteps can be accessed via the results dict.

In [4]:
# Show the analysis settings
pprint(res_basic.results["analysis"])

{'compressOutput': True,
 'dataset': 'C:\\Users\\jmannhardt\\polybox\\PhD\\02-ZEN\\ZEN-garden\\tests\\testcases\\test_1a',
 'discountRate': 0.06,
 'fileFormat': 'csv',
 'folderNameSystemSpecification': 'systemSpecification',
 'folderOutput': 'C:\\Users\\jmannhardt\\polybox\\PhD\\02-ZEN\\ZEN-garden\\outputs',
 'headerDataInputs': {'setCapacityTypes': 'capacityType',
                      'setCarriers': 'carrier',
                      'setConversionTechnologies': 'technology',
                      'setDependentCarriers': 'carrier',
                      'setEdges': 'edge',
                      'setElements': 'element',
                      'setExistingTechnologies': 'existingTechnology',
                      'setInputCarriers': 'carrier',
                      'setLocation': 'location',
                      'setNodes': 'node',
                      'setOutputCarriers': 'carrier',
                      'setScenarios': 'scenario',
                      'setStorageTechnologies': 'tech

### Extracting individual data frames

You can access individual data frames from any component with the `get_df` method. However, the behavior differs slightly for the usecases.
If the optimization was run with perfect foresight and no scenarios, then the data frame is returned as `pandas.Series`. For myopic foresight, the final dataframe is stiched together from the relevant outputs if the component is set yearly. If multiple scenarios were run, a dictionary containing the data frame for all different scenarios is returned. However, you can specify the scenario in advance, then only a single data frame is returned.

Additionally, the method provides a shortcut to save the extracted dataframe directly to a CSV, or in case of different scenarios to a bundle of CSV files.

In [5]:
# This will return a single data frame
df = res_basic.get_df('exportCarrierFlow')
df

carrier      node  timeOperation
heat         CH    0                0.0
             DE    0                0.0
natural_gas  CH    0                0.0
             DE    0                0.0
Name: exportCarrierFlow, dtype: float64

In [6]:
# you can also save the dataframe directly to a csv:
df = res_basic.get_df('carbonEmissionsOvershoot', to_csv="test")
# this prints out the content of the generated csv
!cat test.csv

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
# if we have multiple scenarios, all dataframe will be returned in a dict
df_dict = res_scenario.get_df('exportCarrierFlow')
pprint(df_dict)

{'scenario_': carrier      node  timeOperation
heat         CH    0                0.0
             DE    0                0.0
natural_gas  CH    0                0.0
             DE    0                0.0
Name: exportCarrierFlow, dtype: float64,
 'scenario_2': carrier      node  timeOperation
heat         CH    0                0.0
             DE    0                0.0
natural_gas  CH    0                0.0
             DE    0                0.0
Name: exportCarrierFlow, dtype: float64,
 'scenario_3': carrier      node  timeOperation
heat         CH    0                0.0
             DE    0                0.0
natural_gas  CH    0                0.0
             DE    0                0.0
Name: exportCarrierFlow, dtype: float64}


In [8]:
# saving this to a csv will create a file for each scenario
df_dict = res_scenario.get_df('exportCarrierFlow', to_csv="test2")
# list files and print content of one
!echo "Generated files: "
!ls test2*
!echo 
!echo "Output of test2_scenario_2.csv"
!cat test2_scenario_2.csv

"Generated files: "


'ls' is not recognized as an internal or external command,
operable program or batch file.


ECHO is on.
"Output of test2_scenario_2.csv"


'cat' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# we can also specify a specific scneario
df = res_scenario.get_df('exportCarrierFlow', scenario="scenario_2")
df

carrier      node  timeOperation
heat         CH    0                0.0
             DE    0                0.0
natural_gas  CH    0                0.0
             DE    0                0.0
Name: exportCarrierFlow, dtype: float64

In [8]:
# This will collect the values from the results folders of the different myopic foresight runs
df = res_mf.get_df("carbonEmissionsCumulative")
df

0    149.341143
1    298.682286
2    448.023428
Name: carbonEmissionsCumulative, dtype: float64

### Helper functions

The `Results` class implements some helper functions. The function `getTotal` allows you to calculate the total Value of a component, for a given year, element and scenario if desired. And the function `getFullTS` calculates the full (disaggregate) time series for a given component.

In [9]:
# We read out a normal dataframe
df = res_mf.get_df("inputFlow")
print(df)

technology          carrier      node  timeOperation
natural_gas_boiler  natural_gas  CH    0                  3.875823
                                       1                  3.875823
                                       2                  3.875823
                                       3                 16.909306
                                       4                 16.909306
                                       5                 16.909306
                                       6                 27.856524
                                       7                 27.856524
                                       8                 27.856524
                                       9                 10.738593
                                       10                10.738593
                                       11                10.738593
                                       12                31.583558
                                       13                31.583558
         

In [10]:
# We can calculate the total value of the dataframe
res_mf.getTotal(df)

technology          carrier      node
natural_gas_boiler  natural_gas  CH      3.237618e+05
                                 DE      1.894616e+06
dtype: float64

In [11]:
# and also get the full timeseries (disaggregate) if we want
df_disaggregate = res_mf.getFullTS(df)
df_disaggregate

natural_gas_boiler  natural_gas  CH  0         16.909306
                                     1         20.973992
                                     2         20.973992
                                     3         24.445700
                                     4         31.583558
                                                 ...    
                                 DE  26275    182.559451
                                     26276    182.559451
                                     26277    156.288835
                                     26278    114.846837
                                     26279    114.846837
Length: 52560, dtype: float64

### Some Plots

Here could be some example plots